In [432]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import statistics as stats

In [415]:
# Life expectancy (http://apps.who.int/gho/data/node.main.688?lang=en) - chosen 2015
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHOSIS_000001,WHOSIS_000015&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;SEX"
life = pd.read_csv(url) 
life = life.ix[1:,0:3]
life.columns = ['Country', 'Year', 'LifeExpectancy']
life['Year'] = life['Year'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
life['LifeExpectancy'] = life['LifeExpectancy'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
life = life.loc[life['Year'] == 2015]
del life['Year']

In [416]:
life.head(5)

,Country,LifeExpectancy
1,Afghanistan,60.5
18,Albania,77.8
35,Algeria,75.6
56,Angola,52.4
73,Antigua and Barbuda,76.4


In [417]:
life.dtypes

Country            object
LifeExpectancy    float64
dtype: object

In [425]:
# Smoking status (http://apps.who.int/gho/data/node.main.1250?lang=en) - 2013, only value
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/M_Est_tob_curr,M_Est_tob_daily,M_Est_cig_curr,M_Est_cig_daily&profile=crosstable&filter=COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;SEX"
smoking = pd.read_csv(url)
smoking = smoking.ix[1:,0:3]
del smoking['Unnamed: 1']
smoking.columns = ['Country', 'Smoking']
smoking['Smoking'] = smoking['Smoking'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [426]:
smoking.head(5)

,Country,Smoking
1,Afghanistan,NaN
2,Albania,29.7
3,Algeria,NaN
4,Andorra,33.1
5,Angola,NaN


In [385]:
# Alcohol (http://apps.who.int/gho/data/node.main.A1026?lang=en) - 2010 was used
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/SA_0000001400&profile=crosstable&filter=COUNTRY:*;YEAR:2015;YEAR:2014;YEAR:2013;YEAR:2012;YEAR:2011;YEAR:2010;YEAR:2009;YEAR:2008;YEAR:2007;YEAR:2006;YEAR:2005;YEAR:2004;YEAR:2003;YEAR:2002;YEAR:2001;YEAR:2000&x-sideaxis=COUNTRY;DATASOURCE;ALCOHOLTYPE&x-topaxis=GHO;YEAR"
alcohol = pd.read_csv(url)
del alcohol['Unnamed: 1']
alcohol = alcohol.ix[1:]
alcohol.columns = ['Country', 'AlcType', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
                  '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
alcohol.head(5)
alcohol.isnull().sum() # Finding most complete recent year - 2010 looks like it

Country      0
AlcType      0
2015       964
2014       504
2013       339
2012       342
2011        54
2010        16
2009        14
2008        14
2007        14
2006         9
2005         8
2004        14
2003        14
2002        19
2001        19
2000        18
dtype: int64

In [386]:
alcohol.ix[:, 7:] = alcohol.ix[:, 7:].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
alcohol = pd.melt(alcohol, id_vars=['Country', 'AlcType'], 
                  value_vars=['2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
                              '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000'])

In [387]:
alcohol.columns = ['Country', 'AlcType', 'Year', 'AlcConsumption']
alcohol['Year'] = alcohol['Year'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
alcohol = alcohol.loc[alcohol['Year'] == 2010]
alcohol = alcohol[alcohol['AlcType'].str.contains("All types")]
del alcohol['AlcType']
del alcohol['Year']
alcohol.head(5)

,Country,AlcConsumption
4920,Afghanistan,0.01
4925,Albania,5.28
4930,Algeria,0.45
4935,Andorra,10.60
4940,Angola,7.80


In [333]:
# Overweight (http://apps.who.int/gho/data/node.main.A897A?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_BMI_25A&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;AGEGROUP;SEX"
oweight = pd.read_csv(url)
oweight = oweight.ix[3:,0:2]
oweight.columns = ['Country', 'Overweight']

oweight['Overweight'] = oweight['Overweight'].apply(lambda x: x.split(' [')[0])
oweight['Overweight'] = oweight['Overweight'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [334]:
oweight.head(5)

,Country,Overweight
3,Afghanistan,16.2
4,Albania,52.7
5,Algeria,59.1
6,Andorra,66.9
7,Angola,30.9


In [336]:
# Physical activity (http://apps.who.int/gho/data/node.main.A893?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_PAC,NCD_PAA&profile=crosstable&filter=AGEGROUP:YEARS18-PLUS;COUNTRY:*;SEX:*;&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX"
exercise = pd.read_csv(url)
del exercise['Unnamed: 2']
exercise = exercise.ix[1:, :3]
exercise.columns = ['Country', 'Year', 'LowExercise']
del exercise['Year']

exercise['LowExercise'] = exercise['LowExercise'].apply(lambda x: x.split(' [')[0])
exercise['LowExercise'] = exercise['LowExercise'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [337]:
exercise.head(5)

,Country,LowExercise
1,Afghanistan,NaN
2,Albania,NaN
3,Algeria,34.4
4,Andorra,26.1
5,Angola,NaN


In [338]:
# Cholesterol (http://apps.who.int/gho/data/node.main.A884?lang=en) - 2008
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/CHOL_01,CHOL_02&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX"
chol = pd.read_csv(url)
del chol['Unnamed: 2']
chol = chol.ix[1:, :3]
chol.columns = ['Country', 'Year', 'Cholesterol']
del chol['Year']

chol['Cholesterol'] = chol['Cholesterol'].apply(lambda x: x.split(' [')[0])
chol['Cholesterol'] = chol['Cholesterol'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [339]:
chol.head(5)

,Country,Cholesterol
1,Afghanistan,21.9
2,Albania,45.3
3,Algeria,39.4
4,Andorra,67.1
5,Angola,30.9


In [400]:
# Blood sugar (http://apps.who.int/gho/data/node.main.A869?lang=en) - 2014
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_GLUC_03,NCD_GLUC_04&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX"
bsugar = pd.read_csv(url)
del bsugar['Unnamed: 2']
bsugar = bsugar.ix[1:, :3]
bsugar.columns = ['Country', 'Year', 'BloodSugar']

bsugar['BloodSugar'] = bsugar['BloodSugar'].apply(lambda x: x.split(' [')[0])
bsugar['BloodSugar'] = bsugar['BloodSugar'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
bsugar = bsugar.loc[bsugar['Year'].str.contains("2014")]
del bsugar['Year']

In [401]:
bsugar.head(5)

,Country,BloodSugar
1,Afghanistan,8.4
3,Albania,8.3
5,Algeria,10.5
7,Andorra,9.1
9,Angola,5.6


In [420]:
# Unsafe water, sanitation and hygiene (http://apps.who.int/gho/data/node.main.167?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHS5_122,WHS5_158&profile=crosstable&filter=COUNTRY:*;RESIDENCEAREATYPE:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;RESIDENCEAREATYPE"
sanitation = pd.read_csv(url)
sanitation = sanitation[['Unnamed: 0', 'Unnamed: 1',
                        'Population using improved drinking-water sources (%).2',
                        'Population using improved sanitation facilities (%).2']]
sanitation = sanitation.ix[1:,]
sanitation.columns = ['Country', 'Year', 'ImprovedWater', 'ImprovedSanitation']
sanitation['Year'] = sanitation['Year'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
sanitation['ImprovedWater'] = sanitation['ImprovedWater'].apply(lambda x: pd.to_numeric(x, errors ='coerce'))
sanitation['ImprovedSanitation'] = sanitation['ImprovedSanitation'].apply(lambda x: pd.to_numeric(x, errors ='coerce'))
sanitation = sanitation.loc[sanitation['Year'] == 2015]
del sanitation['Year']

In [421]:
sanitation.head(5)

,Country,ImprovedWater,ImprovedSanitation
1,Afghanistan,55.3,31.9
3,Albania,95.1,93.2
6,Algeria,83.6,87.6
9,Andorra,100.0,100.0
12,Angola,49.0,51.6


In [346]:
# Maternal deaths (http://apps.who.int/gho/data/node.main.MATMORT?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000026,MORT_MATERNALNUM&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO"
maternal = pd.read_csv(url)
maternal = maternal.ix[:, :3]
maternal.columns = ['Country', 'Year', 'MaternalDeaths']

maternal['MaternalDeaths'] = maternal['MaternalDeaths'].str.replace(' ', '')
maternal['MaternalDeaths'] = maternal['MaternalDeaths'].apply(lambda x: x.split('[')[0])
maternal['MaternalDeaths'] = maternal['MaternalDeaths'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

maternal['Year'] = maternal['Year'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
maternal = maternal.loc[maternal['Year'] == 2015]
del maternal['Year']

In [347]:
maternal.head(5)

,Country,MaternalDeaths
0,Afghanistan,396
3,Albania,29
6,Algeria,140
9,Angola,477
12,Argentina,52


In [218]:
# UV radiation exposure (http://apps.who.int/gho/data/node.main.164?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/UV_1&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
uvrad = pd.read_csv(url)
uvrad = uvrad.ix[1:, ]
uvrad.columns = ['Country', 'UVRadiation']

In [219]:
uvrad.head(5)

,Country,UVRadiation
1,Afghanistan,4132
2,Albania,2542
3,Algeria,3253
4,Andorra,2469
5,Angola,5287


In [225]:
# Homicide (http://apps.who.int/gho/data/node.main.VIOLENCEHOMICIDE?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/VIOLENCE_HOMICIDENUM,VIOLENCE_HOMICIDERATE&profile=crosstable&filter=COUNTRY:*;AGEGROUP:-;SEX:-&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
homicides = pd.read_csv(url)
del homicides['Estimates of number of homicides']
homicides = homicides.ix[1:,]
homicides.columns = ['Country', 'HomicideRate']

homicides['HomicideRate'] = homicides['HomicideRate'].apply(lambda x: x.split('[')[0])
homicides['HomicideRate'] = homicides['HomicideRate'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [226]:
homicides.head(5)

,Country,HomicideRate
1,Afghanistan,7.3
2,Albania,5.0
3,Algeria,4.4
4,Andorra,0.8
5,Angola,10.7


In [229]:
# Road traffic deaths (http://apps.who.int/gho/data/node.main.A997?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/RS_196,RS_198&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
traffdeath = pd.read_csv(url)
del traffdeath['Estimated number of road traffic deaths']
traffdeath = traffdeath.ix[1:,]
traffdeath.columns = ['Country', 'TrafficDeaths']

In [230]:
traffdeath.head(5)

,Country,TrafficDeaths
1,Afghanistan,15.5
2,Albania,15.1
3,Algeria,23.8
4,Andorra,7.6
5,Angola,26.9


In [247]:
# Malaria (http://apps.who.int/gho/data/view.main.14111?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MALARIA002&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
malaria = pd.read_csv(url)
malaria = malaria.ix[1:, :2]
malaria.columns = ['Country', 'Malaria']

malaria['Malaria'] = malaria['Malaria'].str.replace(' ', '')
malaria['Malaria'] = malaria['Malaria'].str.replace('&lt;', '')
malaria['Malaria'] = malaria['Malaria'].apply(lambda x: x.split('[')[0])
malaria['Malaria'] = malaria['Malaria'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [248]:
malaria.head(5)

,Country,Malaria
1,Afghanistan,250000.0
2,Algeria,50.0
3,Angola,3400000.0
4,Argentina,0.0
5,Armenia,NaN


In [253]:
# Number of people with HIV (http://apps.who.int/gho/data/node.main.620?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/HIV_0000000001&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
hiv = pd.read_csv(url)
hiv = hiv.ix[1:, :2]
hiv.columns = ['Country', 'HIV']

hiv['HIV'] = hiv['HIV'].str.replace(' ', '')
hiv['HIV'] = hiv['HIV'].str.replace('&lt;', '')
hiv['HIV'] = hiv['HIV'].apply(lambda x: x.split('[')[0])
hiv['HIV'] = hiv['HIV'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [254]:
hiv.head(5)

,Country,HIV
1,Afghanistan,6700.0
2,Albania,NaN
3,Algeria,11000.0
4,Andorra,NaN
5,Angola,300000.0


In [260]:
# Tuberculosis (http://apps.who.int/gho/data/view.main.57040ALL?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000020,TB_e_inc_num,TB_e_inc_tbhiv_100k,TB_e_inc_tbhiv_num&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO"
tb = pd.read_csv(url)
tb = tb[['Country', 'Year',
                        'Incidence of tuberculosis (per 100 000 population per year)']]
tb.columns = ['Country', 'Year', 'Tubercululosis']
tb = tb.loc[tb['Year'] == 2014]
del tb['Year']

tb['Tubercululosis'] = tb['Tubercululosis'].str.replace(' ', '')
tb['Tubercululosis'] = tb['Tubercululosis'].str.replace('&lt;', '')
tb['Tubercululosis'] = tb['Tubercululosis'].apply(lambda x: x.split('[')[0])
tb['Tubercululosis'] = tb['Tubercululosis'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [263]:
tb.head(5)

,Country,Tubercululosis
0,Afghanistan,189.0
25,Albania,19.0
50,Algeria,78.0
75,Andorra,9.2
100,Angola,370.0


In [418]:
# Suicide (http://apps.who.int/gho/data/node.main.MHSUICIDE?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MH_12&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;SEX"
suicide = pd.read_csv(url)
suicide = suicide.ix[2:, :2]
suicide.columns = ['Country', 'Suicide']
suicide['Suicide'] = suicide['Suicide'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [419]:
suicide.head(5)

,Country,Suicide
2,Afghanistan,5.7
3,Albania,5.9
4,Algeria,1.9
5,Angola,13.8
6,Argentina,10.3


In [364]:
def mergeFunc(dataframe1, dataframe2):
    return pd.merge(dataframe1, dataframe2, left_on='Country', 
                    right_on='Country', how='outer')

In [427]:
totaldf = mergeFunc(life, smoking)
for i in [alcohol, oweight, exercise, chol, bsugar, sanitation, maternal,
         uvrad, homicides, traffdeath, malaria, hiv, tb, suicide]:
    totaldf = mergeFunc(totaldf, i)

In [428]:
totaldf.head(5)

,Country,LifeExpectancy,Smoking,AlcConsumption,Overweight,LowExercise,Cholesterol,BloodSugar,ImprovedWater,ImprovedSanitation,MaternalDeaths,UVRadiation,HomicideRate,TrafficDeaths,Malaria,HIV,Tubercululosis,Suicide
0,Afghanistan,60.5,NaN,0.01,16.2,NaN,21.9,8.4,55.3,31.9,396.0,4132.0,7.3,15.5,250000.0,6700.0,189.0,5.7
1,Albania,77.8,29.7,5.28,52.7,NaN,45.3,8.3,95.1,93.2,29.0,2542.0,5.0,15.1,NaN,NaN,19.0,5.9
2,Algeria,75.6,NaN,0.45,59.1,34.4,39.4,10.5,83.6,87.6,140.0,3253.0,4.4,23.8,50.0,11000.0,78.0,1.9
3,Angola,52.4,NaN,7.80,30.9,NaN,30.9,5.6,49.0,51.6,477.0,5287.0,10.7,26.9,3400000.0,300000.0,370.0,13.8
4,Antigua and Barbuda,76.4,NaN,7.84,61.9,NaN,43.0,11.8,97.9,NaN,NaN,5148.0,4.4,6.7,NaN,NaN,7.6,NaN


In [429]:
corr_df = totaldf.corr(method='pearson')
print("--------------- CORRELATIONS ---------------")
print(corr_df.head(corr_df.shape[1]))

--------------- CORRELATIONS ---------------
                    LifeExpectancy   Smoking  AlcConsumption  Overweight  \
LifeExpectancy            1.000000  0.249703        0.449908    0.642331   
Smoking                   0.249703  1.000000        0.228790    0.371351   
AlcConsumption            0.449908  0.228790        1.000000    0.337301   
Overweight                0.642331  0.371351        0.337301    1.000000   
LowExercise               0.329608 -0.055418        0.069692    0.463555   
Cholesterol               0.819122  0.343770        0.558656    0.689507   
BloodSugar                0.375444  0.426404       -0.020947    0.689836   
ImprovedWater             0.769728  0.307961        0.395846    0.593261   
ImprovedSanitation        0.849289  0.330889        0.445358    0.686397   
MaternalDeaths           -0.854801 -0.289982       -0.351171   -0.660800   
UVRadiation              -0.557237 -0.425587       -0.564594   -0.364322   
HomicideRate             -0.189353 -0.24451

In [408]:
corr_df.shape[1]

12

In [434]:
Series.mean(totaldf['Suicide'])
#stats.mean(totaldf['Suicide'])

9.7319767441860474

In [456]:
#print(df.loc[df['B'].isin(['one','three'])])

print(totaldf.loc[totaldf['Country'].isin(['Panama', 'Guatemala', 'Australia'])])

       Country  LifeExpectancy  Smoking  AlcConsumption  Overweight  \
7    Australia            82.8     16.0           10.52        64.0   
66   Guatemala            71.9      NaN            2.14        52.0   
123     Panama            77.8      7.4            6.94        62.2   

     LowExercise  Cholesterol  BloodSugar  ImprovedWater  ImprovedSanitation  \
7           23.8         55.2         7.3          100.0               100.0   
66          13.3         27.7         7.5           92.8                63.9   
123          NaN         42.8         9.0           94.7                75.0   

     MaternalDeaths  UVRadiation  HomicideRate  TrafficDeaths  Malaria  \
7               6.0       3206.0           1.1            5.4      NaN   
66             88.0       5141.0          39.9           19.0  10400.0   
123            94.0       4898.0          19.3           10.0    830.0   

         HIV  Tubercululosis  Suicide  
7        NaN             6.4     10.6  
66   49000.0     

In [457]:
totaldf.ix[totaldf['AlcConsumption'].idxmax()]
totaldf.ix[totaldf['ImprovedWater'].idxmin()]

Country               Papua New Guinea
LifeExpectancy                    62.9
Smoking                            NaN
AlcConsumption                    0.79
Overweight                        60.7
LowExercise                       14.7
Cholesterol                       38.2
BloodSugar                        11.8
ImprovedWater                       40
ImprovedSanitation                18.9
MaternalDeaths                     215
UVRadiation                       5377
HomicideRate                      10.8
TrafficDeaths                     16.8
Malaria                        1.3e+06
HIV                              37000
Tubercululosis                     417
Suicide                           12.4
Name: 124, dtype: object